# SLEAP environment

### Read and pickle the h5 file of interest

In [ ]:
import os
import h5py
import pickle

# Specify the directory containing the HDF5 files
directory = '/ceph/aeon/aeon/code/scratchpad/sleap/multi_point_tracking/multi_animal_CameraTop/predictions_social02'
filename = 'AEON4_2024-02-22T18-00-00_combined.analysis.h5'
filepath = os.path.join(directory, filename)

# Open the HDF5 file in read mode
with h5py.File(filepath, 'r') as f:
    # Extract the tracks
    tracks = f['tracks'][:]
    track_names = f['track_names'][:]
    track_names = [name.decode('utf-8') for name in track_names]
    video_path = f['video_path'][()].decode('utf-8')
    print(track_names)
print(tracks.shape)
data = {'tracks': tracks, 'track_names': track_names, 'video_path': video_path}

filename = 'tracks.pkl'
filepath = os.path.join(directory, filename)

# Pickle the tracks data
with open(filepath, 'wb') as f:
    pickle.dump(data, f)


# Switch to Aeon environment

### Load the data back from the pickle file

In [ ]:
import os
import pickle
import aeon
from aeon.io import video
from aeon.schema.schemas import social02
import pandas as pd
import numpy as np
import re

directory = '/ceph/aeon/aeon/code/scratchpad/sleap/multi_point_tracking/multi_animal_CameraTop/predictions_social02'
filename = 'tracks.pkl'
filepath = os.path.join(directory, filename)

with open(filepath, 'rb') as f:
    data = pickle.load(f)
tracks = data['tracks']
track_names = data['track_names']
video_path = data['video_path']
print(tracks.shape)

### Calculations and extraction of possible tube test frames

In [ ]:
# Indices for keypoints
nose_index = 0  
head_index = 1 
spine1_index = 4
spine2_index = 5
spine3_index = 6 
spine4_index = 7  

# Centroid distances 
centroid_mouse0 = tracks[0, :, spine2_index, :]
centroid_mouse1 = tracks[1, :, spine2_index, :]
centroid_distances = np.linalg.norm(centroid_mouse0 - centroid_mouse1, axis=0)

# Relative spine distances
spine4_mouse0 = tracks[0, :, spine4_index, :]
head_mouse0 = tracks[0, :, head_index, :]
head_mouse1 = tracks[1, :, head_index, :]
relative_distances = np.zeros((2, tracks.shape[3]))
relative_distances[0, :] = np.linalg.norm(spine4_mouse0 - head_mouse0, axis=0)
relative_distances[1, :] = np.linalg.norm(spine4_mouse0 - head_mouse1, axis=0)

# Extremity distances
spine4_mouse1 = tracks[1, :, spine4_index, :]
extremity_distances = np.zeros((4, tracks.shape[3]))
extremity_distances[0, :] = np.linalg.norm(head_mouse0 - head_mouse1, axis=0)
extremity_distances[1, :] = np.linalg.norm(spine4_mouse0 - spine4_mouse1, axis=0)
extremity_distances[2, :] = np.linalg.norm(spine4_mouse0 - head_mouse1, axis=0)
extremity_distances[3, :] = np.linalg.norm(spine4_mouse1 - head_mouse0, axis=0)

# Orientation
# Calculate differences in x and y coordinates
dy_tail_nose = tracks[:, 1, nose_index, :] - tracks[:, 1, spine4_index, :]
dx_tail_nose = tracks[:, 0, nose_index, :] - tracks[:, 0, spine4_index, :]
dy_tail_head = tracks[:, 1, head_index, :] - tracks[:, 1, spine4_index, :]
dx_tail_head = tracks[:, 0, head_index, :] - tracks[:, 0, spine4_index, :]
# Calculate angles: 0 degrees if the mice are facing towards the nest, angles increase counterclockwise
angles_tail_nose = np.degrees(np.arctan2(-dy_tail_nose, dx_tail_nose))
angles_tail_head = np.degrees(np.arctan2(-dy_tail_head, dx_tail_head))
# Adjust angles to be in the range [0, 360)
angles_tail_nose = np.where(angles_tail_nose < 0, angles_tail_nose + 360, angles_tail_nose)
angles_tail_head = np.where(angles_tail_head < 0, angles_tail_head + 360, angles_tail_head)
# When angles_tail_nose is NaN, use angles_tail_head
orientations = np.where(np.isnan(angles_tail_nose), angles_tail_head, angles_tail_nose)

In [ ]:
angle_tolerance = 45
max_distance = 50

# Adjust the orientation of mouse 2
adjusted_orientations = (orientations[1] + 180) % 360

# Condition 1: the mice have opposite orientations, within a certain tolerance
orientation_condition = np.isclose(orientations[0], adjusted_orientations, atol=angle_tolerance)
# Condition 2: the distance between the mice's centroids is less than a certain threshold, ensuring they are close to each other
distance_condition = centroid_distances < max_distance
# Condition 3: relative spine measure, removes cases where mice are side by side
relative_distance_condition = relative_distances[1] > relative_distances[0]
# Condition 4: the mice's tail-to-tail distance is greater than their nose-to-nose distance, removes cases where mice are back-to-back
extremity_distance_condition = extremity_distances[1] > extremity_distances[0]
# Find frames where all conditions are true
possible_tube_test_starts = np.where(np.logical_and.reduce([orientation_condition, distance_condition, relative_distance_condition, extremity_distance_condition]))

### Filter possible tube test frames to only keep those where the mice are both within the corridor ROI

In [ ]:
# Use regex to match the pattern for the root and the two timestamps
metadata_retrieval_matches = re.search(r'(.*?)(\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2}).*(\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2})', video_path)
arena_number_match = re.search(r'AEON(\d)', video_path)
# Extract the root part of the path and the two timestamps
root = metadata_retrieval_matches.group(1)
start_time = pd.to_datetime(metadata_retrieval_matches.group(2), format='%Y-%m-%dT%H-%M-%S')
chunk_time = pd.to_datetime(metadata_retrieval_matches.group(3), format='%Y-%m-%dT%H-%M-%S')
arena_number = arena_number_match.group(1)
# # Option to change the drive
new_drive = "/ceph/aeon"
root = re.sub(r'^.*?:', new_drive, root)

# Retrieve the metadata
metadata = aeon.load(
    root, social02.Metadata, start=start_time, end=chunk_time
)["metadata"].iloc[0]

inner_radius = float(metadata.ActiveRegion.ArenaInnerRadius)
outer_radius = float(metadata.ActiveRegion.ArenaOuterRadius)
center_x = float(metadata.ActiveRegion.ArenaCenter.X)
center_y = float(metadata.ActiveRegion.ArenaCenter.Y)
nest_y1 = float(metadata.ActiveRegion.NestRegion.ArrayOfPoint[1].Y)
nest_y2 = float(metadata.ActiveRegion.NestRegion.ArrayOfPoint[2].Y)
if arena_number == '3':
    entrance_y1 = 547
    entrance_y2 = 573
elif arena_number == '4':
    entrance_y1 = 554
    entrance_y2 = 587

# Create an array of frame numbers
frame_numbers = np.arange(tracks.shape[3])

# Get the x and y coordinates of spine2 for both mice
spine2_x = tracks[:, 0, spine2_index, :]
spine2_y = tracks[:, 1, spine2_index, :]

# Calculate the squared distance from the center of the ROI
dist_squared = (spine2_x - center_x)**2 + (spine2_y - center_y)**2

# Check if the distance is within the squared radii for both mice
within_roi = (inner_radius**2 <= dist_squared) & (dist_squared <= outer_radius**2)

# Check if the mice are in excluded regions
in_excluded_region_nest = (spine2_x > center_x) & ((spine2_y >= nest_y1) & (spine2_y <= nest_y2))
in_excluded_region_entrance = (spine2_x < center_x) & ((spine2_y >= entrance_y1) & (spine2_y <= entrance_y2))

# Update the ROI condition to exclude the specified regions
within_roi = within_roi & ~np.any(in_excluded_region_nest | in_excluded_region_entrance, axis=0)
within_roi_both_mice = np.all(within_roi, axis=0)

# Filter the frame numbers where both mice are within the ROI
frame_numbers_in_roi = frame_numbers[within_roi_both_mice]

# Filter possible tube test frames to only keep those where the mice are both within the corridor ROI
possible_tube_test_starts = np.intersect1d(possible_tube_test_starts, frame_numbers_in_roi)

### Divide possible tube test frames into subarrays of consecutive frames = one possible tube test event

In [ ]:
max_frame_gap = 20
min_tube_test_start_frames = 15

# Divide possible_tube_test_starts into sub_arrays of consecutive frames (allowing for gaps up to a certain max)
diffs = np.diff(possible_tube_test_starts)
indices = np.where(diffs > max_frame_gap)[0]
indices += 1
possible_tube_test_starts = np.split(possible_tube_test_starts, indices)

# Filter sub_arrays to keep only those with more than a certain number of frames
possible_tube_test_starts = [sub_array for sub_array in possible_tube_test_starts if len(sub_array) > min_tube_test_start_frames]

In [ ]:
print(len(possible_tube_test_starts))
print(possible_tube_test_starts)

### Extraction of true tube test events

In [ ]:
fps = 50
search_window_seconds = 1
min_distance = 30
max_distance = 60
vid_export_dir = directory + '/tube_test_videos/'

tube_tests_data = {'start_frame': [], 'start_timestamp' : []}

for subarray in possible_tube_test_starts:
    # Check each possible_tube_test_starts frame interval for tracking errors
    # Skeleton flipping (i.e., tail end being mistaken for head) can lead to false tube test detections
    # Take all orientations in the interval, including frames that did not meet all the tube test start conditions
    all_start_orientations = orientations[:, subarray[0]:subarray[-1]+1]
    # Find how often the mice have the same orientation, within a certain tolerance
    orientation_condition = np.isclose(all_start_orientations[0], all_start_orientations[1], atol=angle_tolerance) 
    count = np.count_nonzero(orientation_condition)
    # Move to the next possible tube test start if skeleton flipping is detected
    if count > 1:
        continue

    # Find the possible tube test end frames
    start_frame = subarray[-1]
    search_window = int(np.ceil(fps*search_window_seconds))
    possible_tube_test_tracks = tracks[:, :, :, start_frame:start_frame + search_window]

    # Condition 1: the mice have the same orientations, within a certain tolerance
    orientation_condition = np.isclose(orientations[0, start_frame:start_frame + search_window], orientations[1, start_frame:start_frame + search_window], atol=angle_tolerance)
    # Condition 2: the distance between the mice's centroids is more than a certain threshold, removes cases where mice are fighting or side-by-side
    min_distance_condition = centroid_distances[start_frame:start_frame + search_window] > min_distance
    # Condition 3: the distance between the mice's centroids is less than a certain threshold, removes cases where mice "teleport" due to tracking errors
    max_distance_condition = centroid_distances[start_frame:start_frame + search_window] < max_distance 
    # Find frames where all conditions are true
    possible_tube_test_ends = start_frame + np.where(np.logical_and.reduce([orientation_condition, min_distance_condition, max_distance_condition]))[0]
    # Add tube test to final table if there are frames where all end conditons are met
    # Note that with more accurate identity tracking, there are some additional end conditions that could be added to reduce false positives (see code below)
    if len(possible_tube_test_ends) > 0:
        start_timestamp = chunk_time + pd.Timedelta(seconds=start_frame/fps)
        tube_tests_data['start_frame'].append(start_frame)
        tube_tests_data['start_timestamp'].append(start_timestamp)
        # Export video of the tube test for checking
        vid_start = start_timestamp - pd.Timedelta(seconds=10)
        vid_end   = start_timestamp + pd.Timedelta(seconds=10)
        frames_info = aeon.load(root, social02.CameraTop.Video, start=vid_start, end=vid_end)
        vid = video.frames(frames_info)
        save_path = vid_export_dir + "AEON" + arena_number + "_CameraTop_" + start_timestamp.strftime('%Y-%m-%dT%H-%M-%S') + ".avi"
        video.export(vid, save_path, fps=50)
        
tube_tests = pd.DataFrame(tube_tests_data)
display(tube_tests)

---
### Other possible versions of "extraction of tube test events" code
1. With extra identity reliant conditions

In [ ]:
fps = 50
search_window_seconds = 1
min_distance = 30
max_distance = 60
movement_threshold = 3

tube_tests_data = {'start_frame': [], 'start_timestamp' : [], 'loser':[]}
needs_checking_data = {'start_frame': [], 'start_timestamp' : []}

for subarray in possible_tube_test_starts:
    # Check each possible_tube_test_starts frame interval for tracking errors
    # Skeleton flipping (i.e., tail end being mistaken for head) can lead to false tube test detections
    # Take all orientations in the interval, including frames that did not meet all the tube test start conditions
    all_start_orientations = orientations[:, subarray[0]:subarray[-1]+1]
    # Find how often the mice have the same orientation, within a certain tolerance
    orientation_condition = np.isclose(all_start_orientations[0], all_start_orientations[1], atol=angle_tolerance) 
    count = np.count_nonzero(orientation_condition)
    # Move to the next possible tube test start if skeleton flipping is detected
    if count > 1:
        continue

    # Find the possible tube test end frames
    start_frame = subarray[-1]
    search_window = int(np.ceil(fps*search_window_seconds))
    possible_tube_test_tracks = tracks[:, :, :, start_frame:start_frame + search_window]

    # Condition 1: the mice have the same orientations, within a certain tolerance
    orientation_condition = np.isclose(orientations[0, start_frame:start_frame + search_window], orientations[1, start_frame:start_frame + search_window], atol=angle_tolerance)
    # Condition 2: the distance between the mice's centroids is more than a certain threshold, removes cases where mice are fighting or side-by-side
    min_distance_condition = centroid_distances[start_frame:start_frame + search_window] > min_distance
    # Condition 3: the distance between the mice's centroids is less than a certain threshold, removes cases where mice "teleport" due to tracking errors
    max_distance_condition = centroid_distances[start_frame:start_frame + search_window] < max_distance 
    # Find frames where all conditions are true
    possible_tube_test_ends = start_frame + np.where(np.logical_and.reduce([orientation_condition, min_distance_condition, max_distance_condition]))[0]
    # Move to the next possible tube test start if no possible tube test end is found
    if len(possible_tube_test_ends) == 0:
        continue

    # Find which mouse turned around (loser)
    start_orientations = orientations[:, subarray]
    start_orientations = np.mean(start_orientations, axis=1)
    end_orientations = orientations[:, possible_tube_test_ends]
    end_orientations = np.mean(end_orientations, axis=1)
    mouse_index = np.argmax(np.abs(start_orientations - end_orientations))
    # Condition 4: the loser is in front of the winner, removes cases where mouse A squeezes past mouse B, and mouse B turns around (false tube test detection)
    mean_tail0_head1_distance = np.mean(extremity_distances[2, possible_tube_test_ends])
    mean_tail1_head0_distance = np.mean(extremity_distances[3, possible_tube_test_ends])
    front_mouse_condition = mean_tail0_head1_distance < mean_tail1_head0_distance if mouse_index == 0 else mean_tail1_head0_distance < mean_tail0_head1_distance
    # Condition 5: the loser's average movement between each frame is larger than a certain threshold, removes cases where the mice are stationary (e.g., grooming)
    points_frame = possible_tube_test_tracks[mouse_index, :, spine2_index, :-1]  # all but the last frame
    points_next_frame = possible_tube_test_tracks[mouse_index, :, spine2_index, 1:]  # all but the first frame
    differences = points_next_frame - points_frame
    mean_movement = np.nanmean(np.linalg.norm(differences, axis=0))
    movement_condition = mean_movement > movement_threshold
    # Conditions 4 and 5 rely on accurate identity tracking: if they are not met, the tube test should be checked manually
    start_timestamp = chunk_time + pd.Timedelta(seconds=start_frame/fps)
    if not front_mouse_condition or not movement_condition:
        vid_export_dir = directory + '/tube_test_videos/needs_checking/'
        needs_checking_data['start_frame'].append(start_frame)
        needs_checking_data['start_timestamp'].append(start_timestamp)
    # Add tube test to final table if all conditons are met
    else:
        vid_export_dir = directory + '/tube_test_videos/'
        tube_tests_data['start_frame'].append(start_frame)
        tube_tests_data['start_timestamp'].append(chunk_time + pd.Timedelta(seconds=start_frame/fps))
        tube_tests_data['loser'].append(track_names[mouse_index])
    # Export video of the tube test or potential tube test in need of checking
    vid_start = start_timestamp - pd.Timedelta(seconds=10)
    vid_end   = start_timestamp + pd.Timedelta(seconds=10)
    frames_info = aeon.load(root, social02.CameraTop.Video, start=vid_start, end=vid_end)
    vid = video.frames(frames_info)
    save_path = vid_export_dir + "AEON" + arena_number + "_CameraTop_" + start_timestamp.strftime('%Y-%m-%dT%H-%M-%S') + ".avi"
    # video.export(vid, save_path, fps=50)
        
tube_tests = pd.DataFrame(tube_tests_data)
needs_checking = pd.DataFrame(needs_checking_data)
display(tube_tests)
display(needs_checking)

2. With identity swapping detection (if no identity swapping detected, identity reliant conditions are checked)

In [ ]:
fps = 50
search_window_seconds = 1
min_distance = 30
max_distance = 60
movement_threshold = 3

tube_tests_data = {'start_frame': [], 'start_timestamp' : [], 'loser':[]}
needs_checking_data = {'start_frame': [], 'start_timestamp' : []}

for subarray in possible_tube_test_starts:
    # Check each possible_tube_test_starts frame interval for tracking errors
    # Skeleton flipping (i.e., tail end being mistaken for head) can lead to false tube test detections
    # Take all orientations in the interval, including frames that did not meet all the tube test start conditions
    all_start_orientations = orientations[:, subarray[0]:subarray[-1]+1]
    # Find how often the mice have the same orientation, within a certain tolerance
    orientation_condition = np.isclose(all_start_orientations[0], all_start_orientations[1], atol=angle_tolerance) 
    count = np.count_nonzero(orientation_condition)
    # Move to the next possible tube test start if skeleton flipping is detected
    if count > 1:
        continue

    # Find the possible tube test end frames
    generate_vid = False
    start_frame = subarray[-1]
    start_timestamp = chunk_time + pd.Timedelta(seconds=start_frame/fps)
    search_window = int(np.ceil(fps*search_window_seconds))
    possible_tube_test_tracks = tracks[:, :, :, start_frame:start_frame + search_window]

    # Condition 1: the mice have the same orientations, within a certain tolerance
    orientation_condition = np.isclose(orientations[0, start_frame:start_frame + search_window], orientations[1, start_frame:start_frame + search_window], atol=angle_tolerance)
    # Condition 2: the distance between the mice's centroids is more than a certain threshold, removes cases where mice are fighting or side-by-side
    min_distance_condition = centroid_distances[start_frame:start_frame + search_window] > min_distance
    # Condition 3: the distance between the mice's centroids is less than a certain threshold, removes cases where mice "teleport" due to tracking errors
    max_distance_condition = centroid_distances[start_frame:start_frame + search_window] < max_distance 
    # Find frames where all conditions are true
    possible_tube_test_ends = start_frame + np.where(np.logical_and.reduce([orientation_condition, min_distance_condition, max_distance_condition]))[0]
    # Move to the next possible tube test start if no possible tube test end is found
    if len(possible_tube_test_ends) == 0:
        continue
    
    # The next steps require accurate identity tracking
    # Check for identity swapping errors from buildup to tube test (first frame in subarray) to end of search window
    centroid_mouse0_trimmed = centroid_mouse0[:, subarray[0]:start_frame + search_window - 1]  # all but the last frame
    centroid_mouse1_trimmed_next_frame = centroid_mouse1[:, subarray[0] + 1:start_frame + search_window] # all but the first frame
    shifted_centroid_dists = np.linalg.norm(centroid_mouse0_trimmed - centroid_mouse1_trimmed_next_frame, axis=0)
    identity_swap = np.isclose(shifted_centroid_dists, 0, atol=10)
    # ---
    # # Code to look at % of frames where idenities are swapped - could be used in place of np.any(identity_swap) condition below
    # true_indices = np.where(identity_swap)[0]
    # swap_indices = true_indices[0::2]
    # swap_back_indices = true_indices[1::2]
    # if len(swap_indices) > len(swap_back_indices): swap_back_indices = np.append(swap_back_indices, len(identity_swap))
    # num_frames_swapped_id = np.sum(swap_back_indices-swap_indices)
    # proportion_frames_swapped_id = num_frames_swapped_id/len(identity_swap)
    # ---
    # If there is identity swapping, the potential tube test needs to be manually checked 
    if np.any(identity_swap):
        generate_vid = True
        vid_export_dir = directory + '/tube_test_videos/needs_checking/'
        needs_checking_data['start_frame'].append(start_frame)
        needs_checking_data['start_timestamp'].append(start_timestamp)
    # If there is no identity swapping, a loser can be identified and automatic checks can ensure the tube test is valid
    else:
        # Find which mouse turned around (loser)
        start_orientations = orientations[:, subarray]
        start_orientations = np.mean(start_orientations, axis=1)
        end_orientations = orientations[:, possible_tube_test_ends]
        end_orientations = np.mean(end_orientations, axis=1)
        mouse_index = np.argmax(np.abs(start_orientations - end_orientations))
        # Condition 4: the loser is in front of the winner, removes cases where mouse A squeezes past mouse B, and mouse B turns around (false tube test detection)
        mean_tail0_head1_distance = np.mean(extremity_distances[2, possible_tube_test_ends])
        mean_tail1_head0_distance = np.mean(extremity_distances[3, possible_tube_test_ends])
        front_mouse_condition = mean_tail0_head1_distance < mean_tail1_head0_distance if mouse_index == 0 else mean_tail1_head0_distance < mean_tail0_head1_distance
        # Condition 5: the loser's average movement between each frame is larger than a certain threshold, removes cases where the mice are stationary (e.g., grooming)
        points_frame = possible_tube_test_tracks[mouse_index, :, spine2_index, :-1]  # all but the last frame
        points_next_frame = possible_tube_test_tracks[mouse_index, :, spine2_index, 1:]  # all but the first frame
        differences = points_next_frame - points_frame
        mean_movement = np.nanmean(np.linalg.norm(differences, axis=0))
        movement_condition = mean_movement > movement_threshold
        # Add tube test to final table if all conditons are met
        if front_mouse_condition and movement_condition:
            generate_vid = True
            vid_export_dir = directory + '/tube_test_videos/'
            tube_tests_data['start_frame'].append(start_frame)
            tube_tests_data['start_timestamp'].append(chunk_time + pd.Timedelta(seconds=start_frame/fps))
            tube_tests_data['loser'].append(track_names[mouse_index])
    if generate_vid:
        # Export video of the tube test or potential tube test in need of checking
        vid_start = start_timestamp - pd.Timedelta(seconds=10)
        vid_end   = start_timestamp + pd.Timedelta(seconds=10)
        frames_info = aeon.load(root, social02.CameraTop.Video, start=vid_start, end=vid_end)
        vid = video.frames(frames_info)
        save_path = vid_export_dir + "AEON" + arena_number + "_CameraTop_" + start_timestamp.strftime('%Y-%m-%dT%H-%M-%S') + ".avi"
        # video.export(vid, save_path, fps=50)
        
tube_tests = pd.DataFrame(tube_tests_data)
needs_checking = pd.DataFrame(needs_checking_data)
display(tube_tests)
display(needs_checking)